In [1]:
import os
import time
import logging
import yaml
import ast
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from pprgo import utils
from pprgo import ppr
from pprgo import pprgo

import igraph
import pickle

In [2]:
# Set up logging
logger = logging.getLogger()
logger.handlers = []
ch = logging.StreamHandler()
formatter = logging.Formatter(
        fmt='%(asctime)s (%(levelname)s): %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')
ch.setFormatter(formatter)
logger.addHandler(ch)
logger.setLevel('INFO')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# !wget --show-progress -O data/reddit.npz https://ndownloader.figshare.com/files/23742119

# Download dataset

# Load config

In [4]:
with open('config_demo.yaml', 'r') as c:
    config = yaml.safe_load(c)

In [5]:
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [6]:
data_file           = config['data_file']           # Path to the .npz data file
data_file = 'data/cora_full.npz'
# data_file = 'data/pubmed.npz'
# data_file = 'data/reddit.npz'
# data_file = 'data/mag_coarse.npz'

split_seed          = config['split_seed']          # Seed for splitting the dataset into train/val/test
ntrain_div_classes  = config['ntrain_div_classes']  # Number of training nodes divided by number of classes
attr_normalization  = config['attr_normalization']  # Attribute normalization. Not used in the paper

alpha               = config['alpha']               # PPR teleport probability
alpha = 0.25
eps                 = config['eps']                 # Stopping threshold for ACL's ApproximatePR
topk                = config['topk']                # Number of PPR neighbors for each node
topk=32
ppr_normalization   = config['ppr_normalization']   # Adjacency matrix normalization for weighting neighbors

hidden_size         = config['hidden_size']         # Size of the MLP's hidden layer
nlayers             = config['nlayers']             # Number of MLP layers
weight_decay        = config['weight_decay']        # Weight decay used for training the MLP
dropout             = config['dropout']             # Dropout used for training

lr                  = config['lr']                  # Learning rate
max_epochs          = config['max_epochs']          # Maximum number of epochs (exact number if no early stopping)
batch_size          = config['batch_size']          # Batch size for training
batch_mult_val      = config['batch_mult_val']      # Multiplier for validation batch size

eval_step           = config['eval_step']           # Accuracy is evaluated after every this number of steps
run_val             = config['run_val']             # Evaluate accuracy on validation set during training

early_stop          = config['early_stop']          # Use early stopping
patience            = config['patience']            # Patience for early stopping

nprop_inference     = config['nprop_inference']     # Number of propagation steps during inference
inf_fraction        = config['inf_fraction']        # Fraction of nodes for which local predictions are computed during inference

In [7]:
# import warnings
# warnings.filterwarnings("ignore")

# Load the data

In [8]:
start = time.time()
(adj_matrix, attr_matrix, labels,
 train_idx, val_idx, test_idx) = utils.get_data(
        f"{data_file}",
        seed=split_seed,
        ntrain_div_classes=ntrain_div_classes,
        normalize_attr=attr_normalization
)
try:
    d = attr_matrix.n_columns
except AttributeError:
    d = attr_matrix.shape[1]
nc = labels.max() + 1
print(nc)

# print('shape attibute matrix: ', attr_matrix.n_rows)
print('Training: ', len(train_idx))
print('Validation: ', len(val_idx))
print('Testing: ', len(test_idx))

print('train_idx: ', train_idx)

time_loading = time.time() - start
print(f"Runtime: {time_loading:.2f}s")

70
Training:  1400
Validation:  14000
Testing:  3400
train_idx:  [   12    16    18 ... 18790 18793 18798]
Runtime: 0.27s


# Preprocessing: Calculate PPR scores

In [9]:
# pip install networkx

In [10]:
from networkx import from_scipy_sparse_matrix, k_truss, core_number

# g_networkX = from_scipy_sparse_matrix(adj_matrix)
# core_numbers2 = core_number(g_networkX)
# np.save('core-numbers-networkx', np.array(list(core_numbers.values())))

g_networkX = None


In [11]:
#Compute core numbers
# core_numbers = np.load('core-numbers-networkx.npy')

# graph = igraph.Graph.Adjacency((adj_matrix.todense()> 0).tolist())
# core_numbers = np.array(graph.coreness())
# try:
#     core_numbers = np.load('coredata/reddit-cores.npy')
#     print('Cores loaded ')
# except:
#     print('No file for core numbers')
#     core_numbers = None
core_numbers = None

In [12]:
# with open('graph.pkl', 'wb') as outp:  # Overwrites any existing file.
#         pickle.dump(graph, outp, pickle.HIGHEST_PROTOCOL)

# with open('graph.pkl', 'rb') as inp:
#     graph = pickle.load(inp)

# shortest_path = graph.get_shortest_paths(0, to=[1,2,3])
# print(shortest_path)

In [13]:
# compute the ppr vectors for train/val nodes using ACL's ApproximatePR

start = time.time()

# topk_train = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk, #core_numbers, g_networkX,
#                                  normalization=ppr_normalization) #, S=1, gamma=0.8)

topk_train, core_topk_train, mean_kn = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, train_idx, topk, core_numbers, g_networkX,
                                 normalization=ppr_normalization, S=1, gamma=0.78)
if run_val:
    topk_val = ppr.topk_ppr_matrix(adj_matrix, alpha, eps, val_idx, topk,
                                   normalization=ppr_normalization)
else:
    topk_val = None

time_preprocessing = time.time() - start
print(f"Runtime: {time_preprocessing:.2f}s")


# print('topk_train', topk_train.shape)
# adj_xd = adj_matrix[train_idx]
# print(adj_xd[0:0 + batch_size].shape)

maxium:  1276
Runtime: 9.33s


In [14]:
import scipy.sparse

# scipy.sparse.save_npz('FwdPush-topk_train.npz', topk_train)

In [15]:
# topk_train = scipy.sparse.load_npz('FwdPush-topk_train.npz')

In [16]:
print(topk_train.shape)
# print(core_topk_train)

(1400, 18800)


# Training: Set up model and train

In [17]:
start = time.time()
tf.reset_default_graph()
tf.set_random_seed(0)

K = 5

#ppr like
TEMP = alpha*(1-alpha)**np.arange(K+1)
TEMP[-1] = (1-alpha)**K
TEMP = TEMP.astype('f')


#negative ppr 
# TEMP = (alpha)**np.arange(K+1)
# TEMP = TEMP/np.sum(np.abs(TEMP))
# TEMP = TEMP.astype('f')

#Random
# bound = np.sqrt(3/(K+1))
# TEMP = np.random.uniform(-bound, bound, K+1)
# TEMP = TEMP/np.sum(np.abs(TEMP))
# TEMP = TEMP.astype('f')


# print(TEMP)
# print(TEMP.dtype)

#random version
gamma = 0.5
# gamma = np.array([0.8, 0.2])
# gamma = gamma.astype('f')

model = pprgo.PPRGo(d, nc, hidden_size, nlayers, lr, gamma, weight_decay, dropout, #adj_matrix, train_idx, TEMP, K, topk_train, intermediate_layer=100,
                    sparse_features=type(attr_matrix) is not np.ndarray)



# print(type(attr_matrix) is not np.ndarray)

2022-08-02 16:26:58 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_91178/1542600667.py:2: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.



2022-08-02 16:26:58 (WARNING): From /var/folders/x5/bhlqr68144x_138bllx2mdwh0000gn/T/ipykernel_91178/1542600667.py:3: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.



In [18]:
sess = tf.compat.v1.Session()
with sess.as_default():
    tf.compat.v1.global_variables_initializer().run()
    nepochs, loss_hist, acc_hist, f1_hist = pprgo.train(
            sess=sess, model=model, attr_matrix=attr_matrix,
            train_idx=train_idx, val_idx=val_idx,
            topk_train=topk_train, topk_val=topk_val, core_topk_train=core_topk_train,
            labels=labels, #adj_matrix=adj_matrix,
            max_epochs=max_epochs, batch_size=batch_size, batch_mult_val=batch_mult_val,
            eval_step=eval_step, early_stop=early_stop, patience=patience)
time_training = time.time() - start
logging.info('Training done.')
print(f"Runtime: {time_training:.2f}s")

gamma:  0.62245935
gamma:  0.6212971
gamma:  0.6203562


2022-08-02 16:27:01 (INFO): Epoch 6, step 20: train 3.96735
2022-08-02 16:27:02 (INFO): Epoch 13, step 40: train 3.39769
2022-08-02 16:27:04 (INFO): Epoch 19, step 60: train 3.05700


gamma:  0.5336528
gamma:  0.5317884
gamma:  0.5299318


2022-08-02 16:27:06 (INFO): Epoch 26, step 80: train 2.74731
2022-08-02 16:27:07 (INFO): Epoch 33, step 100: train 2.02430
2022-08-02 16:27:08 (INFO): Epoch 39, step 120: train 1.79398


gamma:  0.4206935
gamma:  0.41892785
gamma:  0.4171739


...

gamma:  0.15544565
gamma:  0.15516672
gamma:  0.1548843


2022-08-02 16:27:28 (INFO): Epoch 166, step 500: train 0.65257
2022-08-02 16:27:29 (INFO): Epoch 173, step 520: train 0.57881
2022-08-02 16:27:30 (INFO): Epoch 179, step 540: train 0.57509


gamma:  0.13999835
gamma:  0.13976556
gamma:  0.13952984


2022-08-02 16:27:31 (INFO): Epoch 186, step 560: train 0.62367
2022-08-02 16:27:32 (INFO): Epoch 193, step 580: train 0.55290
2022-08-02 16:27:33 (INFO): Epoch 199, step 600: train 0.55561
2022-08-02 16:27:33 (INFO): Training done.


Runtime: 35.09s


In [19]:
# print(model.batch_labels)
# print(model.adjacency_weighted_logits)

# Inference (val and test)

In [20]:
start = time.time()
predictions, time_logits, time_propagation = model.predict(
        sess=sess, adj_matrix=adj_matrix, attr_matrix=attr_matrix, alpha=alpha, #topk_ppr=topk_train,
        nprop=nprop_inference, inf_fraction=inf_fraction,
        ppr_normalization=ppr_normalization)
time_inference = time.time() - start
print(f"Runtime: {time_inference:.2f}s")

Runtime: 0.15s


In [21]:
print(acc_hist)


{'train': [0.150390625, 0.251953125, 0.4122340425531915, 0.5546875, 0.736328125, 0.8218085106382979, 0.833984375, 0.90625, 0.9361702127659575, 0.9453125, 0.974609375, 0.973404255319149, 0.970703125, 0.994140625, 0.9920212765957447, 0.9765625, 0.99609375, 0.9946808510638298, 0.990234375, 0.99609375, 0.9946808510638298, 0.99609375, 0.994140625, 0.9973404255319149, 1.0, 0.99609375, 1.0, 0.99609375, 0.998046875, 0.9946808510638298], 'val': []}


# Collect and print results

In [22]:

acc_train = 100 * accuracy_score(labels[train_idx], predictions[train_idx])
acc_val = 100 * accuracy_score(labels[val_idx], predictions[val_idx])
acc_test = 100 * accuracy_score(labels[test_idx], predictions[test_idx])

f1_train = f1_score(labels[train_idx], predictions[train_idx], average='macro')
f1_val = f1_score(labels[val_idx], predictions[val_idx], average='macro')
f1_test = f1_score(labels[test_idx], predictions[test_idx], average='macro')



#gpu_max_bytes = tf.contrib.memory_stats.MaxBytesInUse()
#gpu_memory = sess.run(gpu_max_bytes)
memory = utils.get_max_memory_bytes()

time_total = time_preprocessing + time_training + time_inference

In [23]:
print(f'''
Accuracy: Train: {acc_train:.1f}%, val: {acc_val:.1f}%, test: {acc_test:.1f}%
F1 score: Train: {f1_train:.3f}, val: {f1_val:.3f}, test: {f1_test:.3f}

Runtime: Preprocessing: {time_preprocessing:.2f}s, training: {time_training:.2f}s, inference: {time_inference:.2f}s -> total: {time_total:.2f}s
Memory: Main: {(memory/1024) / 2**30:.2f}GB
''')

#Memory: Main: {memory / 2**30:.2f}GB, GPU: {gpu_memory / 2**30:.3f}GB


Accuracy: Train: 99.6%, val: 62.0%, test: 62.0%
F1 score: Train: 0.992, val: 0.518, test: 0.531

Runtime: Preprocessing: 9.33s, training: 35.09s, inference: 0.15s -> total: 44.57s
Memory: Main: 0.55GB

